In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime as dt
import importlib

import aux_funcs as aux

# Import test data

In [2]:
weather_test_df = aux.read_file('Weather_test_fulldata.csv')
weather_test_df = aux.convert_timestamp(weather_test_df)
print(weather_test_df.shape)
weather_test_df.head()

Weather_test_fulldata.csv read in 0.2 s
Mem. usage decreased to  2.94 Mb (54.2% reduction)
(280320, 3)


,site_id,timestamp,air_temperature
0,0,2017-01-01 00:00:00,17.796875
1,0,2017-01-01 01:00:00,17.796875
2,0,2017-01-01 02:00:00,16.093750
3,0,2017-01-01 03:00:00,17.203125
4,0,2017-01-01 04:00:00,16.703125


In [6]:
building_df = aux.read_file('building_metadata.csv')
building_df['log_square_feet'] = np.log(building_df['square_feet'])

building_metadata.csv read in 0.1 s
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [4]:
test_df = aux.read_file('test.csv')
# timestamp
test_df = aux.convert_timestamp(test_df)
test_df = aux.split_timestamp(test_df)

test.csv read in 19.2 s
Mem. usage decreased to 596.49 Mb (53.1% reduction)


In [29]:
print(test_df.shape)
test_df.head()

(41697600, 10)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month
0,0,0,0,2017-01-01,0,1,2017,52,6,1
1,1,1,0,2017-01-01,0,1,2017,52,6,1
2,2,2,0,2017-01-01,0,1,2017,52,6,1
3,3,3,0,2017-01-01,0,1,2017,52,6,1
4,4,4,0,2017-01-01,0,1,2017,52,6,1


# Join tables

In [7]:
#merge tables
testbw = test_df.join(building_df, on='building_id', rsuffix = 'r')
testbw.drop(columns=['building_idr','year_built','floor_count'],inplace=True)
print(testbw.shape)
testbw.head()


(41697600, 14)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565


In [8]:
#
testbw = pd.merge(testbw, weather_test_df,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')
print(testbw.shape)
testbw.head()

(41697600, 15)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet,air_temperature
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550,17.796875
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387,17.796875
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700,17.796875
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597,17.796875
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565,17.796875


# Predictions

In [9]:
from sklearn.tree import DecisionTreeRegressor
# read trees
import pickle
with open('tree_of_trees_191023.pkl','rb') as file:
    dtrs = pickle.load(file)

In [10]:
dtrs[0]

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [11]:
features_list=['weeknumber','weekday','hour','log_square_feet','air_temperature']

In [13]:
subgroups = testbw.groupby(['primary_use','meter'])['meter'].count().to_frame()
subgroups

meter
primary_use                   meter         
Education                     0      9408240
                              1      3784320
                              2      2382720
                              3      1226400
Entertainment/public assembly 0      3136080
                              1       770880
                              2       525600
                              3       280320
Food sales and service        0        87600
                              1        70080
                              2        35040
                              3        35040
Healthcare                    0       367920
                              1       227760
                              2       157680
                              3        52560
Lodging/residential           0      2540400
                              1       981120
                              2       788400
                              3       192720
Manufacturing/industrial      0       175200
                              1        17520
                              2        70080
Office                        0      4712880
                              1      2330160
                              2      1384080
                              3       578160
Other                         0       438000
                              1        52560
                              2        17520
Parking                       0       385440
                              1        17520
                              2        35040
Public services               0      2715600
                              1       367920
                              2       175200
                              3       157680
Religious worship             0        52560
                              1        17520
Retail                        0       192720
                              1        35040
Services                      0       157680
                              2        35040
Technology/science            0       105120
                              1        35040
                              2        17520
                              3        17520
Utility                       0        70080
                              1        17520
                              2        35040
Warehouse/storage             0       210240
                              2        17520

In [16]:
# Run predictions
subix = subgroups.index
for k in range(0, len(subgroups)):

    use = subix[k][0]
    meter = subix[k][1]
    
    print(k, use, meter)
    print('   Filter by use and meter...')
    # condition
    cond = (testbw['primary_use']== use) & (testbw['meter']==meter)
    # select features
    print('   Select features and datapoints...')
    X = testbw.loc[cond, features_list]
    #pick tree
    dtr = dtrs[k]
    #predict
    print('   Prediction...')
    testbw.loc[cond, 'log_meter_reading'] = dtr.predict(X)
    print('   Done!')
    print('---------------')
    print('')

0 Education 0
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

1 Education 1
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

2 Education 2
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

3 Education 3
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

4 Entertainment/public assembly 0
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

5 Entertainment/public assembly 1
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

6 Entertainment/public assembly 2
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

7 Entertainment/public assembly 3
   Filter by use and meter...
   Selec

In [27]:
filename = 'submission_01_191025.csv'

In [23]:
# export results

testbw.loc[:,['row_id','meter_reading']].to_csv(filename,index=False)

In [18]:
testbw['log_meter_reading'] = testbw['meter_reading']

In [20]:
testbw['meter_reading'] = np.exp(testbw['log_meter_reading'])-1

In [21]:
testbw.head()

,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet,air_temperature,meter_reading,log_meter_reading
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550,17.796875,255.960052,5.548921
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387,17.796875,53.785711,4.003429
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700,17.796875,4.914400,1.777390
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597,17.796875,196.577028,5.286129
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565,17.796875,1127.589829,7.028724


In [24]:
importlib.reload(aux)

<module 'aux_funcs' from 'C:\\rob\\Kaggle\\ASHRAE\\aux_funcs.py'>

In [28]:
aux.get_lines(filename)

['row_id,meter_reading\n',
 '0,255.96005155910484\n',
 '1,53.78571105827696\n',
 '2,4.914400397169019\n',
 '3,196.57702750026465\n']

In [30]:
testbw.meter_reading.isnull().sum()

0